In [13]:
import argparse
from pathlib import Path

batch_size= 64
d_model = 512
early_stopping_activated = False
early_stop_steps_per_half_clr_cycle = 3
nb_epochs = 60

limit_length= None
use_splitting = True
max_length_from_file = False


base_lr = 10**(-6)
max_lr = 0.0001
momentum = 0.9
half_period_cycle = 3
gamma = 0.8

load_from_backup = False
restore_from_backup = tuple(["model_params","scheduler","optimizer","losses","metrics"])

simple_hyp_params = argparse.Namespace(batch_size= batch_size,d_model = d_model,
early_stopping_activated = early_stopping_activated,
early_stop_steps_per_half_clr_cycle = early_stop_steps_per_half_clr_cycle,
nb_epochs = nb_epochs)

dset_truncation =argparse.Namespace(limit_length= limit_length,
                                    use_splitting = use_splitting,
                                    max_length_from_file = max_length_from_file)


opt_params = argparse.Namespace(base_lr = base_lr,
                                max_lr = max_lr,
                                momentum = momentum,
                                half_period_cycle = half_period_cycle,gamma=gamma)

train_state_control = argparse.Namespace(load_from_backup = load_from_backup,
restore_from_backup = restore_from_backup
)


path_to_root = Path("../../")

paths = argparse.Namespace(path_dataset = "data/french_english_dataset/fra.txt",
path_language_info = "models/language_info.pth",
path_dataset_splitting = "dataset_splitting",
path_model_and_dependencies = "models/sequence_translator_transformer_new.pth"
)

for key,path in paths.__dict__.items():
    paths.__dict__[key] = str(path_to_root.joinpath(path))



In [14]:
from ploomber_engine.ipython import PloomberClient
from ploomber import DAG
from pathlib import Path
from ploomber.products import File

# initialize client
client = PloomberClient.from_path(Path("./training_setup.ipynb"))
from argparse import Namespace

from translation_machine.models import transformer_mod
from translation_machine import sentence_mod

initial_namespace = argparse.Namespace(**{key:globals()[key] for key in ["simple_hyp_params","dset_truncation",
                                                  "opt_params","train_state_control",
                                                  "paths"]})
train_setup = client.get_namespace(initial_namespace.__dict__)
for key,val in train_setup.items():
        globals()[key] = val

Executing cell: 12: 100%|███████████████████████| 19/19 [00:01<00:00, 13.25it/s]


In [16]:
import time
import itertools


start = time.time()
min_lr_search = 10**(-7)
max_lr_search = 0.1
size_experiments_step = 2000
cycle_schedule = 5
method = "exponential"



lrs = model_trainer.lr_range_test(min_lr_search,max_lr_search,size_experiments_step,cycle_schedule,method)

NameError: name 'train_data_loader' is not defined

In [ ]:
tmp = np.mean(np.array(losses_lr_finder_exponential).reshape(-1,5),axis=1)
plt.plot(tmp)